In [1]:
from pathlib import Path
import pandas as pd

In [2]:
log_filepath = Path("/home/anh/catkin_ws/logs/wp-2021-10-09-04-31-06.csv")
summary_filepath = Path("/home/anh/catkin_ws/logs/summary-wp-2021-10-09-04-31-06.csv")

In [35]:
waypoint_df = pd.read_csv(log_filepath, names = ["pos_x", "pos_y", "euler_2", "speed"], usecols = ["pos_x", "pos_y"])
waypoint_df

,pos_x,pos_y
0,2.876615,0.000000
1,2.876609,0.000000
2,2.876611,0.000000
3,2.876603,0.000000
4,2.876604,0.000000
...,...,...
57847,9.146208,0.233430
57848,9.146181,0.233413
57849,9.146192,0.233420
57850,9.146186,0.233416


In [36]:
from visualization_msgs.msg import Marker, MarkerArray

In [38]:
# only consider point if differs from previous point by a significant amount
x_change_min = 0.5
y_change_min = 0.5

## This code below is not very good because if car is slow, multiple small change may add up to big change => important points are skipped
# main_waypoint_df = waypoint_df[(waypoint_df.diff()["pos_x"] > x_change_min) | (waypoint_df.diff()["pos_y"] > y_change_min)]

pos_x, pos_y = waypoint_df.iloc[0]
important_idx = [0]
for i in range(len(waypoint_df)):
    cur_pos_x, cur_pos_y = waypoint_df.iloc[i]
    if abs(cur_pos_x - pos_x) > x_change_min or abs(cur_pos_y - pos_y) > y_change_min:
        important_idx.append(i)
        pos_x, pos_y = cur_pos_x, cur_pos_y

main_waypoint_df = waypoint_df.iloc[important_idx, :]
main_waypoint_df

,pos_x,pos_y
0,2.876615,0.000000
1045,3.377366,0.000000
1321,3.877776,0.000000
1550,4.378952,0.000000
1774,4.880098,0.000000
...,...,...
55582,6.767540,-0.008074
55802,7.267921,-0.008604
56026,7.768537,-0.009203
56243,8.271094,-0.009876


In [39]:
main_waypoint_df.to_csv(summary_filepath, index=None)

In [33]:
main_waypoint_df = pd.read_csv(summary_filepath)

In [34]:
main_waypoint_df

,pos_x,pos_y
0,2.876615,0.000000
1,3.077491,0.000000
2,3.277881,0.000000
3,3.481651,0.000000
4,3.683225,0.000000
...,...,...
334,8.262516,-0.009864
335,8.462642,-0.008227
336,8.663881,0.017078
337,8.864703,0.078249


In [13]:
from visualization_msgs.msg import Marker, MarkerArray
import time

In [14]:
# first method to get the marker array
start = time.time()
marker_array = MarkerArray()
for idx in range(len(main_waypoint_df)):
    x, y = main_waypoint_df.iloc[idx][["pos_x", "pos_y"]]
    marker = Marker()
    marker.header.frame_id = "/global_frame"
    marker.id = idx
    marker.pose.position.x = x;
    marker.pose.position.y = y;
    marker.color.r = 1
    marker.color.g = 0.0
    marker.color.b = 0.0
    marker.color.a = 0.0
    marker.scale.x = 0.1
    marker.scale.y = 0.1
    marker.scale.z = 0.1
    marker_array.markers.append(marker) 
end = time.time()
print(f"Time taken = {end-start}")

Time taken = 3.3054749965667725


In [15]:
# use dataframe to speed things up
start = time.time()
marker_array = MarkerArray()
def make_markers(row):
    marker = Marker()
    marker.header.frame_id = "/global_frame"
    marker.id = row.name
    marker.pose.position.x = row["pos_x"];
    marker.pose.position.y = row["pos_y"];
    marker.color.r = 1
    marker.color.g = 0.0
    marker.color.b = 0.0
    marker.color.a = 0.0
    marker.scale.x = 0.1
    marker.scale.y = 0.1
    marker.scale.z = 0.1
    return marker

marker_array.markers = main_waypoint_df.apply(make_markers, axis=1).tolist()
end = time.time()
print(f"Time taken = {end-start}")

Time taken = 0.33698534965515137


# Get the point at the lookahead distance

In [9]:
main_waypoint_df["trans_pt"] = main_waypoint_df.apply(lambda row: (row["pos_x"], row["pos_y"]), axis=1)
main_waypoint_df

,pos_x,pos_y,trans_pt
0,2.876615,0.000000,"(2.876615, 0.0)"
1,3.077491,0.000000,"(3.077491, 0.0)"
2,3.277881,0.000000,"(3.277881, 0.0)"
3,3.481651,0.000000,"(3.481651, 0.0)"
4,3.683225,0.000000,"(3.683225, 0.0)"
...,...,...,...
334,8.262516,-0.009864,"(8.262516, -0.009864)"
335,8.462642,-0.008227,"(8.462642, -0.008227)"
336,8.663881,0.017078,"(8.663881, 0.017078)"
337,8.864703,0.078249,"(8.864703, 0.078249)"


In [20]:
main_waypoint_df["trans_x"] = main_waypoint_df["trans_pt"].apply(lambda row: row[0])
main_waypoint_df["trans_y"] = main_waypoint_df["trans_pt"].apply(lambda row: row[1])
main_waypoint_df["dist"] = (main_waypoint_df["trans_x"]**2 + main_waypoint_df["trans_y"]**2)**0.5
main_waypoint_df

,pos_x,pos_y,trans_pt,trans_x,trans_y,dist
0,2.876615,0.000000,"(2.876615, 0.0)",2.876615,0.000000,2.876615
1,3.077491,0.000000,"(3.077491, 0.0)",3.077491,0.000000,3.077491
2,3.277881,0.000000,"(3.277881, 0.0)",3.277881,0.000000,3.277881
3,3.481651,0.000000,"(3.481651, 0.0)",3.481651,0.000000,3.481651
4,3.683225,0.000000,"(3.683225, 0.0)",3.683225,0.000000,3.683225
...,...,...,...,...,...,...
334,8.262516,-0.009864,"(8.262516, -0.009864)",8.262516,-0.009864,8.262522
335,8.462642,-0.008227,"(8.462642, -0.008227)",8.462642,-0.008227,8.462646
336,8.663881,0.017078,"(8.663881, 0.017078)",8.663881,0.017078,8.663898
337,8.864703,0.078249,"(8.864703, 0.078249)",8.864703,0.078249,8.865048


In [31]:
lookahead_dist = 0.5

ahead_waypoint_df = main_waypoint_df[main_waypoint_df["trans_x"] > 0]
idx = abs(ahead_waypoint_df["dist"]-lookahead_dist).argmin()
point = ahead_waypoint_df.iloc[idx]
x, y = point[["trans_x", "trans_y"]]
x, y

120


(0.404116, -0.007406)

,pos_x,pos_y,trans_pt,trans_x,trans_y,dist
0,2.876615,0.000000,"(2.876615, 0.0)",2.876615,0.000000,2.876615
1,3.077491,0.000000,"(3.077491, 0.0)",3.077491,0.000000,3.077491
2,3.277881,0.000000,"(3.277881, 0.0)",3.277881,0.000000,3.277881
3,3.481651,0.000000,"(3.481651, 0.0)",3.481651,0.000000,3.481651
4,3.683225,0.000000,"(3.683225, 0.0)",3.683225,0.000000,3.683225
...,...,...,...,...,...,...
334,8.262516,-0.009864,"(8.262516, -0.009864)",8.262516,-0.009864,8.262522
335,8.462642,-0.008227,"(8.462642, -0.008227)",8.462642,-0.008227,8.462646
336,8.663881,0.017078,"(8.663881, 0.017078)",8.663881,0.017078,8.663898
337,8.864703,0.078249,"(8.864703, 0.078249)",8.864703,0.078249,8.865048
